In [ ]:
#### This Notebook is useful to evaluate the embedding on Link Prediction task

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import re
import networkx as nx
import math
from tqdm import tqdm
from gensim.models import Word2Vec
import warnings
from scipy import spatial

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix 

In [ ]:
#### Following Functions will extract important users mentioned or indicated to form nodes or vertices of Matrix
#user name are mentioned in URL of the columns "Twitter Reply to","Twitter Retweet of"
def getUserFromUrl(Urlstr):
    #print(Urlstr)
    tok = Urlstr.split('/')
    return tok[3]


#Declaring some global variables
All_User_for_Graph = set()   # Will consider all the users into account if mentioned tweeted retweeted for all the rows
All_OccuranceList = []
BigList = []
#-----

def GetTweetRetweetfromRow(row):
    All_User_for_Graph.add(row['AUTHOR_ID'])
    lis = []
    if(row["Thread Entry Type"] == 'post'):
        return lis
    else:
        ReTweetList = row["Twitter Retweet of"]
        if(len(ReTweetList) >0):
            for reTweet in ReTweetList:
                if(pd.isnull(reTweet)):
                    pass
                else:
                    name = getUserFromUrl(reTweet)
                    lis.append(name)
                    All_User_for_Graph.add(name)
        ReplyList = row['Twitter Reply to']
        if(len(ReplyList) >0):
            for reply in ReplyList:
                if(pd.isnull(reply)):
                    pass
                else:
                    name = getUserFromUrl(reply)
                    lis.append(name)
                    All_User_for_Graph.add(name)
        return list(set(lis))
                
def AllMentionedUser(row):
    All_User_for_Graph.add(row['AUTHOR_ID'])
    lis = []
    tweetList = row["Snippet"]
    for tweet in tweetList:
        lis.extend(re.findall(r"@(\w+)",str(tweet)))
    All_User_for_Graph.update(lis)
    return list(set(lis))

def AllConnections(row):
    conn_lis = []
    conn_lis.append(row['AUTHOR_ID'])
    conn_lis.extend(row['TweetReTweetUser'])
    conn_lis.extend(row['AllMentioneduser'])
    return list(set(conn_lis))

In [ ]:
df = pd.read_excel(r'C:\Users\SHAHBAZ\Desktop\study mat\research papers\New folder\New folder\20200507_Cardiology_HCP_Data.xlsx')
Embedding_Column_Name = ["AUTHOR_ID", "AUTHOR_NAME","TRANS_AUTHOR_BIO","Account Type","Location Name","Gender","Mentioned Authors","Tags","Twitter Reply to","Twitter Retweet of","Thread Entry Type","Snippet"]
Main_DF = df[Embedding_Column_Name]
Main_DF = Main_DF.groupby('AUTHOR_ID',as_index= False)["AUTHOR_NAME","TRANS_AUTHOR_BIO","Account Type","Location Name","Gender","Mentioned Authors","Tags","Twitter Reply to","Twitter Retweet of","Thread Entry Type","Snippet"].agg(lambda x: list(x))

In [ ]:
#### Making a cleaner dataframe ####
for columName in Embedding_Column_Name[1:]:
    for _,x in Main_DF.iterrows():
        x[columName] = list(set(x[columName]))

In [ ]:
#Getting additional user information from the data:
Main_DF['TweetReTweetUser'] = [GetTweetRetweetfromRow(x) for _,x in Main_DF.iterrows()]
Main_DF['AllMentioneduser'] = [AllMentionedUser(x) for _,x in Main_DF.iterrows()]  
Main_DF['AllConnection'] = [AllConnections(x) for _,x in Main_DF.iterrows()] 
for _,x in Main_DF.iterrows():
    All_OccuranceList.extend(x['AllConnection'])
    BigList.append(x['AllConnection'])
All_User = list(All_User_for_Graph)

In [ ]:
### Cleaning the All Connection column to only accept users once and who is a connection only
def CreateTargetConnection(authorid):
    totalList= []
    for lis in BigList:
        if(lis.count(authorid) > 0):
            totalList.extend(lis)
    k = list(filter((authorid).__ne__, totalList))
    return k 

In [ ]:
Main_DF['Target_Connection'] = [CreateTargetConnection(x['AUTHOR_ID']) for _,x in Main_DF.iterrows()]

In [ ]:
Main_DF = Main_DF.head(4280)

In [ ]:
#### The list that keeps record of all the users whose embeddings are available, so we can calculate similarity score for them
All_user_Node = []
for _,x in Main_DF.iterrows():
    All_user_Node.append(x['AUTHOR_ID'])

In [ ]:
### Generate the test data. Important to Note taking into account only users who have atleast 3 other users in their neighbor
## Also sub dividing to mini list of users and their neighbours in order to avoid huge computation of all the user nodes.
Test_EdgeList = []
Test_MainUsers = []
Test_NeighborUsers = []
def GeneratetestedgeList(node,connectionlist):
    connection_list_user = []
    connection_copy = connectionlist.copy()
    for users in connection_copy:
        if(All_user_Node.count(users) > 0):
            connection_list_user.append(users)
    if(len(connection_list_user) >= 3):
        n = math.ceil( 0.5 * len(connection_list_user))
        #random.Random(3).shuffle(connection_list_user)
        l = random.sample(connection_list_user,n)
        Test_MainUsers.append(node)
        for inode in l:
            temp = [node]
            temp.append(inode)
            if(Test_EdgeList.count(temp) == 0):
                Test_NeighborUsers.append(inode)
                Test_EdgeList.append(temp)
                connection_copy.remove(inode)
    return connection_copy

In [ ]:
New_DF = Main_DF[["AUTHOR_ID","AllConnection","Target_Connection"]]

In [ ]:
New_DF["new_Target_Connection"] = [GeneratetestedgeList(x['AUTHOR_ID'],x['Target_Connection']) for _,x in New_DF.iterrows()]

In [ ]:
Test_NeighborUsers = list(set(Test_NeighborUsers))

In [ ]:
### Checking if all the test data nodes have embeddings or not.
present = 0
notpresent  = 0
for edge in Test_EdgeList:
    a,b = edge[0],edge[1]
    if((Test_MainUsers.count(a) > 0) and (Test_NeighborUsers.count(b) > 0)):
        present = present + 1
    else:
        notpresent = notpresent + 1        

In [ ]:
Test_EdgeList

In [ ]:
def ReadMatfromPath(fileName):
    ReadFilepath = "C:/Users/SHAHBAZ/ZS UserEmbedding/BERT Embedding/" + str(fileName) + "_Matrix.txt"
    UserEmb = np.loadtxt(ReadFilepath)
    return UserEmb
def CombineMatrix(Mat1,Mat2):
    Mat = np.zeros((Mat1.shape[1] + Mat2.shape[1]))
    for i in range (1,Mat1.shape[0]):
        vec = []
        vec = np.concatenate((vec,Mat1[i]), axis=None)
        vec = np.concatenate((vec,Mat2[i]), axis=None)
        Mat = np.vstack((Mat,vec))
    return Mat   

In [ ]:
##### creating deep walk embedding #####

In [ ]:
### Creating graph and adding nodes###
G = nx.Graph()
for user in All_User:
    G.add_node(user)
    
### Connecting the edges for users who are linked ####
for _,x in New_DF.iterrows():
    source = x['AUTHOR_ID']
    lis = x['new_Target_Connection']
    
    for target in lis:
        G.add_edge(source, target)
        G[source][target]['weight'] = 1

In [ ]:
def get_randomwalk(node,path_length):
    random_walk = [node]
    for i in range(path_length-1):
        temp = list(G.neighbors(node))
        temp = list(set(temp) - set(random_walk))    
        if len(temp) == 0:
            break

        random_node = random.choice(temp)
        random_walk.append(random_node)
        node = random_node
        
    return random_walk

In [ ]:
#### Implementing random walk ####
all_nodes = list(G.nodes())

random_walks = []
for n in tqdm(all_nodes):
    for i in range(10):
        random_walks.append(get_randomwalk(n,10))
warnings.filterwarnings('ignore')

In [ ]:
# train skip-gram (word2vec) model
model = Word2Vec(window = 4, sg = 1, size = 100, hs = 0,negative = 10,alpha=0.03, min_alpha=0.0007,seed = 14)
#size of 100,200 and 300 dimensions were tested and out of which 100 dimension performed best in MultiView
model.build_vocab(random_walks, progress_per=2)
model.train(random_walks, total_examples = model.corpus_count, epochs=20, report_delay=1)

In [ ]:
DeepWalkMat = np.zeros(100)
for _,x in New_DF.iterrows():
    DeepWalkMat = np.vstack((DeepWalkMat,model[x['AUTHOR_ID']]))
DeepWalkMat = DeepWalkMat[1:,:]

In [ ]:
########Creating Node2Vec Embedding #########
p = 1
q = 1 # p,q are the probabilties
alias_nodes = {}
alias_edges = {}
#Main part that generates the walk
def node2vec_walk(walk_length, start_node):
    walk = [start_node]
    while len(walk) < walk_length:
        cur = walk[-1]
        cur_nbrs = sorted(G.neighbors(cur))
        if len(cur_nbrs) > 0:
            if len(walk) == 1:
                walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
            else:
                prev = walk[-2]
                next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0],alias_edges[(prev, cur)][1])]
                walk.append(next)
        else:
            break

    return walk

def simulate_walks(num_walks, walk_length):
    walks = []
    nodes = list(G.nodes())
    print ('Walk iteration:')
    for walk_iter in range(num_walks):
        print (str(walk_iter+1), '/', str(num_walks))
        random.Random(7).shuffle(nodes)
        for node in nodes:
            walks.append(node2vec_walk(walk_length=walk_length, start_node=node))

    return walks

#give probabilty to edges on basis of p and q ( p is for dfs and q is for bfs)
def get_alias_edge(src, dst):
    unnormalized_probs = []
    for dst_nbr in sorted(G.neighbors(dst)):
        if dst_nbr == src:
            unnormalized_probs.append(G[dst][dst_nbr]['weight']/p)
        elif G.has_edge(dst_nbr, src):
            unnormalized_probs.append(G[dst][dst_nbr]['weight'])
        else:
            unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
    norm_const = sum(unnormalized_probs)
    normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

    return alias_setup(normalized_probs)


In [ ]:
def alias_setup(probs):
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)
    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()
        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)
    return J, q
#set values to nodes and edges
def preprocess_transition_probs():
    
    for node in G.nodes():
        unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
        norm_const = sum(unnormalized_probs)
        normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
        alias_nodes[node] = alias_setup(normalized_probs)

    for edge in G.edges():
        alias_edges[edge] = get_alias_edge(edge[0], edge[1])
        alias_edges[(edge[1], edge[0])] = get_alias_edge(edge[1], edge[0])
        
def alias_draw(J, q):
    K = len(J)
    kk = int(np.floor(np.random.rand()*K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [ ]:
preprocess_transition_probs()
walks = simulate_walks(10,10)

In [ ]:
#skip gram model to generate embedding from walks
model = Word2Vec(window = 10, sg = 1, size = 100, hs = 0,negative = 10,alpha=0.03, min_alpha=0.0007,seed = 14)
#size of 100,200 and 300 dimensions were tested and out of which 100 dimension performed best in MultiView
model.build_vocab(walks, progress_per=2)
model.train(walks, total_examples = model.corpus_count, epochs=20, report_delay=1)

In [ ]:
Node2VecMat = np.zeros(100)
for _,x in New_DF.iterrows():
    Node2VecMat = np.vstack((Node2VecMat,model[x['AUTHOR_ID']]))
Node2VecMat = Node2VecMat[1:,:]

In [ ]:
### this function calculate the mean similarity score of a node with its neighbors
def CalculateNeighbourMeanSimilarity(node,neighbourList,Matrix):
    count = 0
    simi_sum = 0
    rowd = New_DF.loc[New_DF['AUTHOR_ID'] == node]
    node_ind = rowd.index.values.astype(int)[0]
    node_vec = Matrix[node_ind]
    
    for neigh_node in neighbourList:
        rowd = New_DF.loc[New_DF['AUTHOR_ID'] == neigh_node]
        if(len(rowd['AUTHOR_ID']) > 0):
            neigh_ind = rowd.index.values.astype(int)[0]
            neigh_vec = Matrix[neigh_ind]
            res = 1 - spatial.distance.cosine(node_vec,neigh_vec)
            simi_sum = simi_sum + abs(res)
            count = count + 1
    if(count > 0):
        return simi_sum / count
    else:
        return 0.8       

In [ ]:
def non_neighbors(graph, node):
    nbors = list(set(graph.neighbors(node)))   #| set([node])
    lis = []
    for nnode in Test_NeighborUsers:
        if(nbors.count(nnode) == 0):
            lis.append(nnode)
    return lis

In [ ]:
## Calculate the number of edges possible between unconnected nodes
def non_edges(graph):
    if graph.is_directed():
        for u in Test_MainUsers:   #modified
            for v in non_neighbors(graph, u):
                #yield (u, v)
                l = [u,v]
                return l
    else:
        S = []
        count = 1
        for u in Test_MainUsers:
            for v in non_neighbors(graph, u):
                l = [u,v]
                if(S.count(l) == 0):
                    S.append(l)
            if((count % 10) == 0):
                print("Epoch = " + str(count) + " / " + str(len(Test_MainUsers)))
            count = count + 1
        return S

In [ ]:
ebunch = non_edges(G)

In [ ]:
len(ebunch)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []

In [ ]:
### method to evaluate if the link between 2 nodes should be established using resource allocation score and then using
## The cosine similarity score between the nodes and establishment is done only when the similarity score is above 
## a multiplicative factor of nodes neighbor hood mean score. The method takes embedding matrix and cutoff score as input.
def predictlink_resourceAllocation(usr1,usr2,Matrix,cutoff_sc):
    score = sum(1 / G.degree(w) for w in nx.common_neighbors(G, usr1, usr2))
    if(score > 0):
        rowd_usr1 = New_DF.loc[New_DF['AUTHOR_ID'] == usr1]
        usr1_sim = rowd_usr1['Mean_Neighbour_Similarity']
        usr1_ind = rowd_usr1.index.values.astype(int)[0]
        rowd_usr2 = New_DF.loc[New_DF['AUTHOR_ID'] == usr2]
        usr2_sim = rowd_usr2['Mean_Neighbour_Similarity']
        usr2_ind = rowd_usr2.index.values.astype(int)[0]
        
        usr1_vec = Matrix[usr1_ind]
        usr2_vec = Matrix[usr2_ind]
        
        res = 1 - spatial.distance.cosine(usr1_vec,usr2_vec)
        if(abs(res) > (cutoff_sc * max(usr1_sim.values[0],usr2_sim.values[0]))):
            temp1 = [usr1,usr2]
            corr_pred_list_resource.append(temp1)

In [ ]:
### method to evaluate if the link between 2 nodes should be established using Jaccard Coefficient score and then using
## The cosine similarity score between the nodes and establishment is done only when the similarity score is above 
## a multiplicative factor of nodes neighbor hood mean score. The method takes embedding matrix and cutoff score as input.
def predict_link_JaccardCoeff(usr1,usr2,Matrix,cutoff_sc):
    cnbors = list(nx.common_neighbors(G, usr1, usr2))
    union_size = len(set(G[usr1]) | set(G[usr2]))
    score = len(cnbors) / union_size
    if(score > 0):
        rowd_usr1 = New_DF.loc[New_DF['AUTHOR_ID'] == usr1]
        usr1_sim = rowd_usr1['Mean_Neighbour_Similarity']
        usr1_ind = rowd_usr1.index.values.astype(int)[0]
        rowd_usr2 = New_DF.loc[New_DF['AUTHOR_ID'] == usr2]
        usr2_sim = rowd_usr2['Mean_Neighbour_Similarity']
        usr2_ind = rowd_usr2.index.values.astype(int)[0]
        
        usr1_vec = Matrix[usr1_ind]
        usr2_vec = Matrix[usr2_ind]
        
        res = 1 - spatial.distance.cosine(usr1_vec,usr2_vec)
        if(abs(res) > (cutoff_sc * max(usr1_sim.values[0],usr2_sim.values[0]))):
            temp1 = [usr1,usr2]
            corr_pred_list_jacard.append(temp1)

In [ ]:
def CompleteLinkPrediction_Resource(Matrix,cutoff_sc):    
    #ebunch = non_edges(G) # calculating the non-edges
    count_alo = 1
    for t in ebunch:  # was list around ebunch
        predictlink_resourceAllocation(t[0],t[1],Matrix,cutoff_sc)
        if(count_alo % 50000 == 0):
            print("Epoch Resource Allocation " + str (count_alo) + " / " + str(len(ebunch)))
        count_alo = count_alo + 1
    corr_pred = 0
    for test_edge in Test_EdgeList:
        if(corr_pred_list_resource.count(test_edge) > 0):
            corr_pred = corr_pred + 1
    print("edges predicted =" + str(len(corr_pred_list_resource)))
    print("Precision " + str(corr_pred / len(Test_EdgeList)))
    print("Recall " + str(corr_pred / len(corr_pred_list_resource)))
    print("Resource allocation correctly predicted = "+str(corr_pred) + " / " + str(len(Test_EdgeList)) + " accuracy= " + str(corr_pred/len(Test_EdgeList)))

In [ ]:
def CompleteLinkPrediction_Jaccard(Matrix,cutoff_sc):   
    #ebunch = non_edges(G) # calculating the non-edges
    count = 1
    for t in ebunch:
        predict_link_JaccardCoeff(t[0],t[1],Matrix,cutoff_sc)
        if(count % 50000 == 0):
            print("Epoch Resource Jaccard " + str (count) + " / " + str(len(ebunch)))
        count = count + 1
    corr_pred = 0
    for test_edge in Test_EdgeList:
        if(corr_pred_list_jacard.count(test_edge) > 0):
            corr_pred = corr_pred + 1
    print("edges predicted =" + str(len(corr_pred_list_jacard)))
    recall = corr_pred / len(Test_EdgeList)
    prec = corr_pred / (len(corr_pred_list_jacard) - len(Test_EdgeList) + corr_pred) 
    print("Precision " + str(prec))    
    print("Recall " + str(recall))
    f1score = (2 * prec * recall) / (prec + recall)
    print("F1 score " + str(f1score))
    print("Jaccard Coefficient correctly predicted = "+str(corr_pred) + " / " + str(len(Test_EdgeList)) + " accuracy= " + str(corr_pred/len(Test_EdgeList)))

In [ ]:
### The method that use previous defined methods to print the result takes embeding matrix and cutoff score as input
def CompleteEval(Matrix,cutoff_sc):
    New_DF['Mean_Neighbour_Similarity'] = ""
    for _,x in New_DF.iterrows():
        x['Mean_Neighbour_Similarity'] = CalculateNeighbourMeanSimilarity(x['AUTHOR_ID'],x['new_Target_Connection'],Matrix)
    print("Mean similarity for users calculated")
    corr_pred_list_resource = []
    corr_pred_list_jacard = []
    
    CompleteLinkPrediction_Jaccard(Matrix,cutoff_sc)
    #CompleteLinkPrediction_Resource(Matrix,cutoff_sc)    

In [ ]:
postDoc_600 = ReadMatfromPath('PostDoc_Basic')
postDoc_600 = postDoc_600[:4280,:]

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(postDoc_600,0.8)

In [ ]:
ClinicalBertSnippetEmbMat = ReadMatfromPath('ClinalBERT_SnippetEmbedding')
ClinicalBertAuthorInfoEmbMat = ReadMatfromPath('ClinalBERT_Autho_Info_Embedding')
ClinicalBertCombinedEmbMat = ReadMatfromPath('ClinalBERT_Combined_AutoEnc_Emb')

ClinicalBertCombinedEmbMat = ClinicalBertCombinedEmbMat[1:,:]
ClinicalBertSnippetEmbMat = ClinicalBertSnippetEmbMat[1:,:]
ClinicalBertAuthorInfoEmbMat = ClinicalBertAuthorInfoEmbMat[1:,:]

ClinicalBERT_1536_Mat = CombineMatrix(ClinicalBertSnippetEmbMat,ClinicalBertAuthorInfoEmbMat)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(ClinicalBERT_1536_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(ClinicalBertCombinedEmbMat,0.8)

In [ ]:
######Scibert ############
SciBertSnippetEmbMat = ReadMatfromPath('SciBERT_SnippetEmbedding')
SciBertAuthorInfoEmbMat = ReadMatfromPath('SciBERT_Autho_Info_Embedding')
SciBertCombinedEmbMat = ReadMatfromPath('SciBERT_Combined_Embedding')

SciBertSnippetEmbMat = SciBertSnippetEmbMat[1:,:]
SciBertAuthorInfoEmbMat = SciBertAuthorInfoEmbMat [1:,:]
SciBertCombinedEmbMat = SciBertCombinedEmbMat [1:,:]
SciBert_combine_1536 = CombineMatrix(SciBertSnippetEmbMat,SciBertAuthorInfoEmbMat)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(SciBert_combine_1536,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(SciBertCombinedEmbMat,0.8)

In [ ]:
############ creating data frame for a single node #####
#ebunch  #ASabanayagamMD
nodeA = []
nodeB = []
similarityScore = []
Jaccard_coeeficient = []
Actual_connection = []
count = 1
for lis in ebunch:
    if (lis[0] == 'ASabanayagamMD'):
        nodeA.append(lis[0])
        nodeB.append(lis[1])
        cnbors = list(nx.common_neighbors(G, lis[0], lis[1]))
        union_size = len(set(G[lis[0]]) | set(G[lis[1]]))
        score = len(cnbors) / union_size
        Jaccard_coeeficient.append(score)
        rowd_usr1 = New_DF.loc[New_DF['AUTHOR_ID'] == lis[0]]
        usr1_ind = rowd_usr1.index.values.astype(int)[0]
        rowd_usr2 = New_DF.loc[New_DF['AUTHOR_ID'] == lis[1]]
        usr2_ind = rowd_usr2.index.values.astype(int)[0]
        
        usr1_vec = postDoc_600[usr1_ind]
        usr2_vec = postDoc_600[usr2_ind]
        
        res = 1 - spatial.distance.cosine(usr1_vec,usr2_vec)
        similarityScore.append(abs(res))
        if(Test_EdgeList.count(lis) > 0):
            Actual_connection.append(1)
        else:
            Actual_connection.append(0)
    if(count % 50000 == 0):
        print("Epoch " + str(count) + " / 266090")
    count = count + 1

In [ ]:
data = {'main node': nodeA, 'neighbor node': nodeB, 'cosine similarity': similarityScore, 'Jaccard coefficient score': Jaccard_coeeficient,'Actual_connection':Actual_connection}
df = pd.DataFrame(data, columns = ['main node','neighbor node','cosine similarity','Jaccard coefficient score','Actual_connection'])

In [ ]:
df.to_csv('SingleNodeScore.csv',header=True,index=False)

In [ ]:
############3end################

In [ ]:
BioBertSnippetEmbMat = ReadMatfromPath('BioBERT_SnippetEmb')
BioBertAuthorInfoEmbMat = ReadMatfromPath('BioBERT_Autho_Info_Embedding')
BioBERT_1536_Mat = CombineMatrix(BioBertSnippetEmbMat,BioBertAuthorInfoEmbMat)
BioBERT_1536_Mat = BioBERT_1536_Mat[1: , :]
BioBert_reduced_300 = ReadMatfromPath('BioBERT_Combined_AutoEnc_Emb')
BioBert_reduced_300 = BioBert_reduced_300[1:,:]

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(BioBERT_1536_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(BioBert_reduced_300,0.8)

In [ ]:
ClinicalBERT_1636_DW_Mat = CombineMatrix(ClinicalBERT_1536_Mat,DeepWalkMat)
ClinicalBERT_400_DW_Mat = CombineMatrix(ClinicalBertCombinedEmbMat,DeepWalkMat)
SciBERT_1636_DW_Mat = CombineMatrix(SciBert_combine_1536,DeepWalkMat)
SciBERT_400_DW_Mat = CombineMatrix(SciBertCombinedEmbMat,DeepWalkMat)
BioBERT_1636_DW_Mat = CombineMatrix(BioBERT_1536_Mat,DeepWalkMat)
BioBERT_400_DW_Mat = CombineMatrix(BioBert_reduced_300,DeepWalkMat)

ClinicalBERT_1636_Node2Vec_Mat = CombineMatrix(ClinicalBERT_1536_Mat,Node2VecMat)
ClinicalBERT_400_Node2Vec_Mat = CombineMatrix(ClinicalBertCombinedEmbMat,Node2VecMat)
SciBERT_1636_Node2Vec_Mat = CombineMatrix(SciBert_combine_1536,Node2VecMat)
SciBERT_400_Node2Vec_Mat = CombineMatrix(SciBertCombinedEmbMat,Node2VecMat)
BioBERT_1636_Node2Vec_Mat = CombineMatrix(BioBERT_1536_Mat,Node2VecMat)
BioBERT_400_Node2Vec_Mat = CombineMatrix(BioBert_reduced_300,Node2VecMat)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(ClinicalBERT_1636_DW_Mat,0.8)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(ClinicalBERT_400_DW_Mat,0.8)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(SciBERT_1636_DW_Mat,0.8)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(SciBERT_400_DW_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(BioBERT_1636_DW_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(BioBERT_400_DW_Mat,0.8)

In [ ]:
corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(ClinicalBERT_1636_Node2Vec_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(ClinicalBERT_400_Node2Vec_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(SciBERT_1636_Node2Vec_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(SciBERT_400_Node2Vec_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(BioBERT_1636_Node2Vec_Mat,0.8)

corr_pred_list_resource = []
corr_pred_list_jacard = []
CompleteEval(BioBERT_400_Node2Vec_Mat,0.8)